# Deep Learning with Pytorh step-by-step

## Simple Regression Problem

In [9]:
import numpy as np 
import torch 
import torch.optim as optim
import torch.nn as nn 

## Data generation

In [18]:
true_b = 1
true_w = 2 
N = 100

# set the random seed for numpy 
np.random.seed(43)

x= np.random.rand(N,1)
epsilon = (.1 * np.random.rand(N,1))

y = true_b + true_w *x + epsilon

### Generating training and validaiton sets

In [19]:
# Generate training and validating sets 
idx = np.arange(N)
np.random.shuffle(idx)

# Use first 80 randowm indices for train
train_idx = idx[:int(N*.8)]
val_idx = idx[int(N*.8):]

# Generate train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

## Full Pipeline

### Data preparation

In [20]:
%%writefile ./data_preparation.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Tranform the data from numpy array to torch tensor
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

Overwriting ./data_preparation.py


In [21]:
%run -i ./data_preparation.py

### Model Config

In [47]:
%%writefile ./model_configuration.py
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        #Make `b` and `w` be the parameters of the model
        #Wrap them with `nn.Parameter` 
        #self.b = nn.Parameter(torch.randn(1,
        #                                requires_grad=True,
        #                                dtype=torch.float))
        #self.w = nn.Parameter(torch.randn(1,
        #                                requires_grad=True,
        #                                dtype=torch.float))
    def forward(self, x):
        # Compute the outputs /predictions
        return self.linear(x)

# Sets learning rate
lr = 0.1 

# Step 0 : Initialize parameters 'b' and 'w' randomly
torch.manual_seed(42)

# Create a model and send it at once to the device
model = ManualLinearRegression().to(device) # 1)

# Define a SGD optimizer to update the parameters
#optimizer = torch.optim.SGD([b,w], lr=lr)
optimizer = torch.optim.SGD(model.parameters(),lr=lr)


#Define a MSE loss function 
loss_fn = nn.MSELoss(reduction="mean")  


Overwriting ./model_configuration.py


In [48]:
%run -i ./model_configuration.py

### Model training

In [49]:
%%writefile ./model_training.py
# Define number of epochs 
n_epochs=1000

for epoch in range(n_epochs): 
    model.train()  #2)
    # Step 1: Computes the model's predicted output - forward pass
    # No more manula prediction 
    #yhat = b + w*x_train_tensor 
    yhat = model(x_train_tensor)  #3)
    
    # Step 2: Computes the loss
    # No more manual loss
    # error = (yhat - y_train_tensor)
    # loss = (error**2).mean()
    loss = loss_fn(yhat, y_train_tensor) #2
    
    # Step 3: Computes gradients for both 'b' and 'w' parameters
    loss.backward()
    
    # Step 4: Updates parameters using gradients and the learning rate
    # No more manual update
    # with torch.no_grade():
    #    b-=lr*b.grad
    #    w-=lr*w.grad
    optimizer.step()
    
    
    # Graident Zeroing
    # No more telling pytorch to let gradients go 
    #b.grad.zero()
    #w.grad.zero()
    optimizer.zero_grad()



Overwriting ./model_training.py


In [50]:
%run -i ./model_training.py

## Model parameters

In [51]:
#print(b,w)
#Inspect the parameters using its state_dict
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[1.9947]])), ('linear.bias', tensor([1.0557]))])


In [54]:
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9947]], requires_grad=True), Parameter containing:
tensor([1.0557], requires_grad=True)]


In [55]:
x = torch.randn(10, 1)

In [56]:
x

tensor([[ 0.4740],
        [ 0.1978],
        [ 1.1561],
        [ 0.3965],
        [-2.4661],
        [ 0.3623],
        [ 0.3765],
        [-0.1808],
        [ 0.3930],
        [ 0.4327]])